In [7]:
inputFolder=""
folderCounter=1

In [4]:
# debug cell
with open('simple_table.txt', 'r') as f:
    print(type(f))
    cnt = 2
    for i,line in enumerate(f):
        print(i, ':\t', line)
        cnt -= 1
        if cnt == 0:
            break

<class '_io.TextIOWrapper'>
0 :	 CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO:AF	UKB-b-6134:ES	UKB-b-6134:SE	UKB-b-6134:LP	UKB-b-6134:AF	UKB-b-6134:SS	UKB-b-6134:EZ	UKB-b-6134:SI	UKB-b-6134:NC	UKB-b-6134:ID

1 :	 1	49298	rs10399793	T	C	.	PASS	0.623238	0.0017795	0.00366743	0.200659	0.623238	.	.	.	.	rs10399793



In [ ]:
# debug cell
import gzip
with gzip.open('./ukb-b-6134.vcf.gz') as f:
    print(f)

In [ ]:
import os
import json
import pickle
import time
import gzip
import shutil

def generate_gwas_output_as_tsv_file(inVCF_File, writeFile=None):
    root = './tabular_format_gwas_data/'
    t0 = time.time()
    if writeFile is None:
        writeFile = root + inVCF_File.split('/')[-1].split('.')[0] + '.tsv'
    
    print("Processing:", inVCF_File)
    if ".gz" == inVCF_File[-3:]:
        with gzip.open(inVCF_File, 'rb') as f_in:
            with open(root+'temp_vcf.vcf', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        inVCF_File = root+'temp_vcf.vcf'
    print("gzip open done :", inVCF_File)
    """
    Convert all the VCF into simplified tsv format
    Source: https://github.com/everestial/VCF-simplify#table-of-contents
    python3 ./VCF-Simplify/VCF-Simplify/VcfSimplify.py SimplifyVCF -toType table -inVCF ./ukb-b-6134.vcf -outFile ./simple_table.txt
    """
    vcf_simplify_path = "./VCF-Simplify/VCF-Simplify/VcfSimplify.py"
    out_File = root+'temp_table.tsv'
    os.system(f"python3 {vcf_simplify_path} SimplifyVCF -toType table -inVCF {inVCF_File} -outFile {out_File}")
    print(writeFile)
        
    f_read = open(out_File, 'r')
    f_write = open(writeFile, 'w')
    cnt = 0
    for line in f_read:
#         print(line)
        newliner = ''
        if '\n' == line[-1]:
            newliner = '\n'
            line = line.strip()
        if 'CHROM' in line:
            line = line.upper() + '\tPVAL_generated_from_LP'
        else:
            LP = float(line.split('\t')[10])
#             LP = float(line.split('\t')[14])
            #find out LP column from the tsv file
            
            p_val = str(10**(-1 * LP))
            line = line + '\t' + p_val
        line = line + newliner
    #     print(line)
        f_write.write(line)
        cnt += 1
    #     if cnt == 20: break

    f_read.close()
    f_write.close()
    print(f'Total SNPs: {cnt} | Total exec time: {time.time() - t0} seconds')

In [ ]:

generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-20289.vcf.gz', writeFile=None)
generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-19732.vcf.gz', writeFile=None)

# generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-17006.vcf.gz', writeFile=None)
# generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-5779.vcf.gz', writeFile=None)
# generate_gwas_output_as_tsv_file(inVCF_File='./tabular_format_gwas_data/ukb-b-15541.vcf.gz', writeFile=None)


# need to just specify the file path here
# ebi-a-GCST005920.vcf.gz
# ebi-a-GCST005923.vcf.gz
# ukb-b-14699.vcf.gz
# ukb-b-323.vcf.gz
# ukb-b-6358
# next update in the 'GWAS_datasets_to_consider.xlsx'


In [ ]:

# ' abc def\n'.strip()

In [ ]:
# 10**(-1*0.200659) #0.6300007

In [8]:
# %pip install openpyxl
import pandas as pd

category_map = {
    'Continuous' : 'beta', 
    'Categorical Ordered (assumed continuous)': 'beta',
    'Binary': 'or',
    'NA (Possibly binary)': 'or'
}

df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
print(df['Category'].unique())
GWAS_ID = 'ieu-b-111'
category = category_map[df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]] 
category

['Continuous' 'Binary' 'Risk factor' 'NA (Possibly binary)'
 'Categorical Ordered (assumed continuous)']


'beta'

In [9]:
from glob import glob
df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
# all_gwas_vcf_files = glob('/mnt/c/Users/HP/Downloads/Compressed/GWAS/*')
# ROOT = '/mnt/c/Users/HP/Downloads/Compressed/GWAS/'
# ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - {'bbj-a-46', 'bbj-a-78', 'ieu-a-1283', 'ukb-a-257','ukb-b-2209','ukb-b-17627','ieub-109','ieu-b-110','ieu-b-111','met-d-Total_C'})
# bbj-a-78,ukb-a-257,bbj-a-46
# ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - { 'ieu-a-1283','bbj-a-46', 'bbj-a-78'})
ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - { 'ieu-a-1283','bbj-a-46', 'bbj-a-78','ukb-b-14180','ukb-b-13806','ukb-b-12064','ukb-b-323','ukb-b-14699','ukb-b-14699','ukb-b-5779'})
print(len(ALL_GWAS_IDS))

# ukb-b-323
# ukb-b-14699

ukb-b-17243
ukb-b-6358
ukb-b-17006
ukb-b-5779
ukb-b-15541
ukb-b-20289
ukb-b-19732

base_files = {}

# base_files=[] #naeem mod
for GWAS_ID in ALL_GWAS_IDS:

#     file = ROOT + GWAS_ID + '.vcf.gz'
    base_files[GWAS_ID] = category_map[df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]]

#     generate_gwas_output_as_tsv_file(inVCF_File=file, writeFile=None)
# print(name_mapping)
# print(name_mapping)


29


In [ ]:
# base_files
print(base_files)
for GWAS_ID in base_files:
    print(GWAS_ID)
    

In [10]:
gender_map = {'Female': 0,'Male': 1}
ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
def get_gender_and_age(PTID):
    gender = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['PTGENDER'].dropna().unique()[0]
    age = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['AGE'].dropna().unique()[0]
    return str(gender_map[gender]) + ' ' + str(age)

# NUM_TRAINING_SAMPLES = int(830 * .7)
# print('NUM_TRAINING_SAMPLES:', NUM_TRAINING_SAMPLES)
# f_writable = open('./COVAR_FILE.txt', 'w')
# f_TRAINING_SAMPLES = open('./TRAINING_SAMPLES.txt', 'w')
# with open('/mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/covar_mds.txt') as f:
#     for i, line in enumerate(f):
#         if 'FID IID' in line:
#             line = line[:-1] + ' GENDER AGE\n'
#         else:
#             line = line[:-1] + ' ' + get_gender_and_age(PTID=line.split(' ')[1]) + '\n'
#         if i < NUM_TRAINING_SAMPLES:
#             f_TRAINING_SAMPLES.write(' '.join(line.split(' ')[:2])+'\n')
#         f_writable.write(line)
# #         print(line)
# f_writable.close()

/tmp/ipykernel_10821/1785225935.py:2: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  ADNIMERGE = pd.read_csv('ADNIMERGE.csv')


In [ ]:
def run_prsice(GWAS_ID, redo=False):
#     TARGET_DATA = '/mnt/c/Users/HP/Documents/GWA_tutorial/2_Population_stratification/ADNI' # old and smaller dataset
    TARGET_DATA = '../larger_dataset/larger_dataset' # new and larger dataset
    '''
    return_status --> 0: success | 256: Error. Execusion Halted | 2: 
    '''
#     need to alter between these two if needed
    if os.path.isfile(f'./PRSice_output/{GWAS_ID}/'):
#     if True:
        return_status = os.system(f'mkdir ./PRSice_output/{GWAS_ID}/')
        print('0:', return_status)
        if return_status == 256 and not redo:
            return return_status
    print('stat',{base_files[GWAS_ID]})
#     prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
#         --dir ./PRSice_output \
#         --prsice ./PRSice_linux/PRSice_linux \
#         --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
#         --target {TARGET_DATA} \
#         --thread 2 \
#         --stat {base_files[GWAS_ID].upper()} \
#         --{base_files[GWAS_ID]} \
#         --binary-target F \
#         --quantile 10 \
#         --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
#         --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
#         --score std \
#         --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
#         --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
    if os.path.isfile(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'):
        prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
            --dir ./PRSice_output \
            --prsice ./PRSice_linux/PRSice_linux \
            --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
            --target {TARGET_DATA} \
            --thread 2 \
            --stat {base_files[GWAS_ID].upper()} \
            --{base_files[GWAS_ID]} \
            --binary-target F \
            --quantile 10 \
            --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
            --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
            --score std \
            --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
            --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
    else :
        prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
            --dir ./PRSice_output \
            --prsice ./PRSice_linux/PRSice_linux \
            --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
            --target {TARGET_DATA} \
            --thread 2 \
            --stat {base_files[GWAS_ID].upper()} \
            --{base_files[GWAS_ID]} \
            --binary-target F \
            --quantile 10 \
            --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
            --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
            --score std \
            --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt '
#         print(prsice_command)
#         print('stat',{base_files[GWAS_ID]})
#         return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
#         print('1:', return_status)
#         return_status = os.system('echo "===== Done ====="')
#         print('2:', return_status) 

    #             running second time with .valid extraction
#         prsice_command = f'Rscript ./PRSice_linux/PRSice.R \
#             --dir ./PRSice_output \
#             --prsice ./PRSice_linux/PRSice_linux \
#             --base ./tabular_format_gwas_data/{GWAS_ID}.tsv \
#             --target {TARGET_DATA} \
#             --thread 2 \
#             --stat {base_files[GWAS_ID].upper()} \
#             --{base_files[GWAS_ID]} \
#             --binary-target F \
#             --quantile 10 \
#             --out ./PRSice_output/{GWAS_ID}/{GWAS_ID} \
#             --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat {GWAS_ID.upper()}:ES --pvalue PVAL_generated_from_LP \
#             --score std \
#             --cov ../larger_dataset/COVAR_FILE_bigger_dataset.txt \
#             --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'
        

# #     return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # generate the file named {GWAS_ID}.valid
# #     return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
# #     print('1a:', return_status)
# #     return
# #     if os.path.isfile(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid'): 
# #         prsice_command += f' --extract ./PRSice_output/{GWAS_ID}/{GWAS_ID}.valid' 
# #         return_status = os.system(prsice_command + '  --keep ./TRAINING_SAMPLES.txt') # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #         return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #         print('1b:', return_status)
# #         if return_status != 0:
# # #             return_status = os.system(prsice_command) # perform the regression analysis on the training samples only and choose the best p-value-threshold
# #             print('1c:', return_status)
# #     return_status = os.system(prsice_command + '  --all-score --no-regress') # Do not perform the regression analysis and simply output all PRS.
# #     return_status = os.system(prsice_command + '  --all-score') # Do not perform the regression analysis and simply output all PRS.
# #     print('2:', return_status)
#     # in later steps, select the PRS values for the selected p-value-threshold for all the samples (both training and testing)
    print(prsice_command)
    print('stat',{base_files[GWAS_ID]})
    return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
    print('1:', return_status)
    return_status = os.system('echo "===== Done ====="')
    print('2:', return_status) 
#     os.system(f'mv ./prsice_output/{GWAS_ID}.* ./prsice_output/{GWAS_ID}/')
# https://github.com/adiamb/Remove-duplicate-snps-plink

In [ ]:
# here for some files a weird error occurs: "reporter not initialized"
#         bbj-a-78
# ukb-a-257
# bbj-a-46
for GWAS_ID in base_files:
    if GWAS_ID == 'ukb-b-19732' :
        print('*****GWAS ID IS*****',GWAS_ID)
        run_prsice(GWAS_ID=GWAS_ID)
#         problem -> they do not have corresponding entries in prsice_output folder->create those folders manually

In [ ]:
# run_prsice(GWAS_ID='ukb-b-17627')

In [ ]:
!ls 'tabular_format_gwas_data/{GWAS_ID}.tsv'

In [3]:
# # need to remove this
# from glob import glob
# import pandas as pd
# category_map = {
#     'Continuous' : 'beta', 
#     'Categorical Ordered (assumed continuous)': 'beta',
#     'Binary': 'or',
#     'NA (Possibly binary)': 'or'
# }

# df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
# # all_gwas_vcf_files = glob('/mnt/c/Users/HP/Downloads/Compressed/GWAS/*')
# # ROOT = '/mnt/c/Users/HP/Downloads/Compressed/GWAS/'
# # ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - {'bbj-a-46', 'bbj-a-78', 'ieu-a-1283', 'ukb-a-257','ukb-b-2209','ukb-b-17627','ieub-109','ieu-b-110','ieu-b-111','met-d-Total_C'})
# # bbj-a-78,ukb-a-257,bbj-a-46
# # ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - { 'ieu-a-1283','bbj-a-46', 'bbj-a-78'})
# ALL_GWAS_IDS = list(set(df['GWAS_ID'].to_numpy().tolist()) - { 'ieu-a-1283','bbj-a-46', 'bbj-a-78','ukb-b-13806','ukb-b-12064','ukb-b-323','ukb-b-14699','ukb-b-14699','ukb-b-5779'})
# print(len(ALL_GWAS_IDS))

# ALL_GWAS_IDS= {'ukb-b-17006','ukb-b-5779','ukb-b-15541','ukb-b-20289','ukb-b-19732','ukb-b-6324','ukb-b-7663','ukb-b-770','ukb-b-323','ukb-b-14699','ukb-b-14180','ukb-b-17243','ukb-b-6358','','ukb-b-8476','','ukb-b-6134','ukb-b-4424','ukb-b-3957','ukb-b-2209','ukb-b-19953','ukb-b-18275','ukb-b-17627','ukb-b-12064','ukb-a-257','ieu-b-39','ieu-b-38','ieu-b-25','ieu-b-111','ieu-b-110','ieu-b-109','met-d-Total_C','ukb-d-20405_2','ukb-d-20405_1','ukb-d-20405_0','ukb-b-13806','ukb-b-13806','ukb-b-12064','ukb-b-323','ukb-b-14699','ukb-b-14699','ukb-b-5779'}

# base_files = {}

# # base_files=[] #naeem mod
# for GWAS_ID in ALL_GWAS_IDS:

# #     file = ROOT + GWAS_ID + '.vcf.gz'
#     base_files[GWAS_ID] = category_map[df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 2]]

# #     generate_gwas_output_as_tsv_file(inVCF_File=file, writeFile=None)
# # print(name_mapping)
# # print(name_mapping)

# gender_map = {'Female': 0,'Male': 1}
# ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
# def get_gender_and_age(PTID):
#     gender = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['PTGENDER'].dropna().unique()[0]
#     age = ADNIMERGE[ADNIMERGE['PTID'] == PTID]['AGE'].dropna().unique()[0]
#     return str(gender_map[gender]) + ' ' + str(age)


29


IndexError: index 0 is out of bounds for axis 0 with size 0

In [11]:
base_files
GWAS_ID = 'ukb-b-18275'

#====

def get_prs_values(GWAS_ID):
    prsice_output = f'./PRSice_output/{GWAS_ID}/'

    if False:
        with open(prsice_output+f'{GWAS_ID}.all_score', 'r') as f:
            lines = f.readlines()
            lines = ['\t'.join(x.split()) for x in lines]
            lines = '\n'.join(lines)

        with open(prsice_output+f'{GWAS_ID}.all_score.tsv', 'w') as f:
            f.write(lines)

        # best_p_val_threshold = '0.00025005'
        best_p_val_threshold = str(open(prsice_output+f'{GWAS_ID}.summary').readlines()[1].split('\t')[2]) 
    #     print(best_p_val_threshold) 
    prs_array = pd.read_csv(prsice_output+f'{GWAS_ID}.best', ' ')['PRS'].to_numpy() 
    return prs_array

In [12]:
get_prs_values(GWAS_ID=GWAS_ID).shape[0]

/tmp/ipykernel_10821/1595037025.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  get_prs_values(GWAS_ID=GWAS_ID).shape[0]


1816

In [13]:
import os
folderCounter=folderCounter+1
inputFolder="input/"+str(folderCounter)+"/"
os.mkdir(inputFolder)


In [14]:
import numpy as np
import os.path
import json
name_mapping={}
# GWAS_ID
# ukb-b-13806
# ukb-b-12064
# ukb-b-323
# ukb-b-14699
# ukb-b-5779
# base_files=base_files-
print(base_files)



PRS_feature_matrix = np.zeros([len(base_files), 1816])
for i, GWAS_ID in enumerate(base_files):
    print(GWAS_ID)
    prsice_output = f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best'
    if os.path.exists(prsice_output):  
        PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
        name_mapping[GWAS_ID] =df[df['GWAS_ID'] == GWAS_ID].to_numpy()[0, 1]
    else :
        print("no path exists")
with open(inputFolder+"traits_map.json", "w") as outfile:
    json.dump(name_mapping, outfile)

    

{'ukb-b-17006': 'or', 'ukb-b-19953': 'beta', 'ukb-b-18275': 'or', 'ukb-b-8476': 'or', 'ieu-b-38': 'beta', 'ieu-b-25': 'beta', 'met-d-Total_C': 'beta', 'ukb-b-6134': 'beta', 'ukb-b-7663': 'or', 'ukb-a-257': 'or', 'ukb-b-17243': 'or', 'ukb-b-17627': 'beta', 'ukb-b-770': 'beta', 'ukb-b-14180': 'or', 'ukb-b-19732': 'or', 'ukb-d-20405_2': 'or', 'ieu-b-110': 'beta', 'ukb-b-6358': 'or', 'ukb-b-15541': 'or', 'ukb-b-2209': 'beta', 'ukb-b-4424': 'beta', 'ukb-b-3957': 'beta', 'ukb-d-20405_1': 'or', 'ieu-b-111': 'beta', 'ukb-b-6324': 'beta', 'ieu-b-39': 'beta', 'ukb-b-20289': 'or', 'ieu-b-109': 'beta', 'ukb-d-20405_0': 'or'}
ukb-b-17006
ukb-b-19953
ukb-b-18275
ukb-b-8476
ieu-b-38
ieu-b-25
met-d-Total_C
ukb-b-6134
ukb-b-7663
ukb-a-257
ukb-b-17243
ukb-b-17627
ukb-b-770
ukb-b-14180
ukb-b-19732
ukb-d-20405_2
ieu-b-110
ukb-b-6358
ukb-b-15541
ukb-b-2209
ukb-b-4424
ukb-b-3957
ukb-d-20405_1
ieu-b-111
ukb-b-6324
ieu-b-39
ukb-b-20289
ieu-b-109
ukb-d-20405_0


/tmp/ipykernel_10821/3477325959.py:21: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_10821/3477325959.py:21: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_10821/3477325959.py:21: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_10821/3477325959.py:21: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  PRS_feature_matrix[i, :] = get_prs_values(GWAS_ID=GWAS_ID)
/tmp/ipykernel_10821/3477325959.py:21: FutureWar

In [15]:
PRS_feature_matrix = PRS_feature_matrix.T
np.save(inputFolder+'PRS_feature_matrix', PRS_feature_matrix)

In [16]:
PRS_feature_matrix.shape
import random

In [17]:
TRAIN_SAMPLE_SIZE = 550
train_samples = list(range(847))
random.shuffle(train_samples)
train_samples, test_samples = train_samples[:TRAIN_SAMPLE_SIZE], train_samples[TRAIN_SAMPLE_SIZE:]
print(test_samples)
print(len(train_samples), len(test_samples), len(train_samples) + len(test_samples))
PRS_feature_matrix[train_samples].mean(0), PRS_feature_matrix[train_samples].std(0)

[598, 495, 475, 779, 96, 41, 63, 313, 733, 730, 611, 719, 244, 290, 787, 165, 123, 359, 628, 232, 512, 67, 38, 77, 210, 79, 477, 630, 131, 755, 435, 286, 491, 676, 774, 464, 344, 709, 455, 813, 170, 107, 307, 348, 3, 506, 195, 543, 46, 773, 316, 293, 276, 172, 353, 801, 454, 843, 306, 627, 28, 692, 326, 83, 296, 829, 738, 532, 782, 297, 211, 103, 416, 267, 814, 111, 821, 460, 552, 722, 599, 659, 605, 178, 520, 413, 704, 310, 731, 429, 656, 250, 637, 503, 791, 508, 321, 759, 525, 294, 159, 820, 652, 188, 811, 147, 341, 84, 682, 529, 175, 143, 789, 265, 567, 763, 22, 562, 395, 661, 36, 632, 243, 431, 23, 434, 634, 716, 662, 528, 18, 702, 169, 74, 568, 799, 712, 593, 98, 354, 448, 612, 622, 385, 25, 168, 419, 234, 80, 403, 680, 377, 318, 384, 246, 119, 91, 558, 298, 156, 95, 177, 141, 762, 328, 619, 104, 703, 330, 665, 471, 197, 420, 142, 303, 425, 603, 663, 87, 174, 222, 640, 772, 615, 766, 182, 388, 417, 162, 406, 324, 88, 521, 115, 202, 24, 633, 432, 688, 623, 643, 585, 746, 554, 333, 

(array([-0.09385751,  0.01642564, -0.57333424, -0.66106535,  0.00131472,
         0.03094232, -0.11054116, -0.00799823, -0.5540036 , -0.50335698,
        -0.24328287,  0.02036986,  0.0576942 , -0.64526496, -0.51393001,
        -0.46627079, -0.10406896, -0.16907315, -0.09523837, -0.01321587,
         0.10857182, -0.02037406, -0.22436656,  0.04812461, -0.02526823,
        -0.00238182, -0.07277892,  0.03169299, -0.51310047]),
 array([0.98651528, 0.99511254, 0.59826146, 0.7143822 , 0.98329353,
        0.96474891, 1.01554322, 1.01980298, 0.57304096, 0.56689474,
        0.84786868, 1.01617222, 0.99887723, 0.67401163, 0.53247184,
        0.52390114, 0.96567579, 0.87065851, 1.0110494 , 1.04206796,
        0.69504082, 0.94342726, 0.87025147, 0.98745743, 1.04149467,
        0.9915582 , 0.99609098, 0.98166574, 0.59081424]))

In [18]:
PRS_feature_matrix[test_samples].mean(0), PRS_feature_matrix[test_samples].std(0)

(array([-0.07978971, -0.04640348, -0.60723057, -0.69161648,  0.04919969,
        -0.05904626,  0.00680603,  0.00854154, -0.59186978, -0.55326592,
        -0.18246585, -0.02527786, -0.01935378, -0.63260243, -0.55928396,
        -0.55387493, -0.03058033, -0.21428949, -0.08246443, -0.0556736 ,
         0.15217239,  0.07322877, -0.28980899, -0.05651939,  0.06657292,
        -0.00114854, -0.11247226,  0.04622745, -0.53352569]),
 array([0.98330645, 1.02766439, 0.54846196, 0.67247871, 0.94408693,
        0.96591924, 1.06640148, 0.97092508, 0.51717341, 0.52451309,
        0.86483669, 0.85458492, 0.97064907, 0.66244159, 0.50791763,
        0.49968656, 0.99889027, 0.9036183 , 0.99061671, 0.90999983,
        0.64950242, 0.95844263, 0.86128375, 0.89625143, 0.88096603,
        0.98820967, 1.03655526, 1.02930485, 0.55202692]))

In [19]:
PRS_feature_matrix[:].mean(0), PRS_feature_matrix[:].std(0)

(array([ 4.53524247e-11,  1.11084739e-11, -5.24669764e-11,  6.10582832e-11,
         7.99063703e-12, -2.72191628e-11,  1.19856782e-11, -2.76734093e-12,
         9.72282264e-12, -1.15251910e-12, -6.37114248e-12,  1.85110097e-11,
         5.42070353e-12,  3.52919065e-11,  6.98566804e-12, -1.24741255e-11,
        -3.06668484e-11, -8.16188233e-12,  3.02863363e-12,  5.59416361e-11,
        -5.24559427e-11,  3.21943862e-11, -4.03398744e-11, -2.29878925e-11,
        -1.95044098e-11,  7.19383295e-11,  3.05832070e-11,  8.02313086e-12,
        -7.23843658e-11]),
 array([0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463, 0.99972463,
        0.99972463, 0.99972463, 0.99972463, 0.99972463]))

In [20]:
import json
import pandas as pd
GWAS_ID = 'ieu-b-25'
usable_samples_ADNI = pd.read_csv(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best', ' ')['IID'].to_numpy().tolist()
usable_samples_ADNI = {'_'.join(sample.split('_')[1:]):idx for idx,sample in enumerate(usable_samples_ADNI)}
# json.dump(usable_samples_ADNI, open('./usable_samples_ADNI.json', 'w'))
type(usable_samples_ADNI), len(usable_samples_ADNI)

/home/naeem/anaconda3/envs/ad_venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3397: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


(dict, 1816)

In [21]:
!ls PRSice_output

'25_7_22_thyroid traits.zip'   met-d-Total_C   ukb-b-18275   ukb-b-6324
 bbj-a-46		       ukb-a-257       ukb-b-19732   ukb-b-6358
 bbj-a-78		       ukb-b-12064     ukb-b-19953   ukb-b-7663
 ieu-b-109		       ukb-b-13806     ukb-b-20289   ukb-b-770
 ieu-b-110		       ukb-b-14180     ukb-b-2209    ukb-b-8476
 ieu-b-111		       ukb-b-14699     ukb-b-323     ukb-d-20405_0
 ieu-b-25		       ukb-b-15541     ukb-b-3957    ukb-d-20405_1
 ieu-b-38		       ukb-b-17006     ukb-b-4424    ukb-d-20405_2
 ieu-b-39		       ukb-b-17243     ukb-b-5779
 lib			       ukb-b-17627     ukb-b-6134


In [22]:
import json
import numpy as np
# PRS_feature_matrix = np.load('./PRS_feature_matrix.npy')
PRS_feature_matrix = np.load(inputFolder+'PRS_feature_matrix.npy')
# usable_samples_ADNI = json.load(open('./usable_samples_ADNI.json'))
print(PRS_feature_matrix.shape, usable_samples_ADNI.__len__())

# usable_samples_ADNI

(1816, 29) 1816


In [24]:
THRESHOLD_MONTH = 12*4
ADNIMERGE = pd.read_csv('ADNIMERGE.csv')
print(ADNIMERGE.head())
print(ADNIMERGE.shape)
ADNI_TEMP = ADNIMERGE[['RID', 'PTID', 'DX', 'Month']].dropna().sort_values(by=['PTID', 'Month'])
Dementia = ADNI_TEMP[ADNI_TEMP['DX']=='Dementia']
print("Dementia shape : ", Dementia.shape)
# print(Dementia)
Samples_Dementia = set(Dementia['PTID'].unique())
print("Samples Dementia length : ", len(Samples_Dementia) )
Samples_NonDementia = set(ADNI_TEMP['PTID'].unique()) - Samples_Dementia
print("Samples NonDementia length : ", len( Samples_NonDementia ) )

# Samples_NonDementia
Final_Samples_Dementia = set()
for sample in Samples_Dementia:
    last_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['DX'].iloc[-1]
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
#     if True:
    if last_month_of_dx >= THRESHOLD_MONTH :
#         if last_dx != 'Dementia':
#             print(last_dx)
        if last_dx == 'Dementia' and sample in usable_samples_ADNI:
#             why check for dementia again?
                Final_Samples_Dementia = Final_Samples_Dementia.union({sample})

Final_Samples_NonDementia = set()
for sample in Samples_NonDementia:
    last_month_of_dx = ADNI_TEMP[ADNI_TEMP['PTID']==sample]['Month'].iloc[-1]
#     if last_month_of_dx >= THRESHOLD_MONTH and sample in usable_samples_ADNI:
    if sample in usable_samples_ADNI:
        Final_Samples_NonDementia = Final_Samples_NonDementia | {sample} # union symbol in python sets
    
# sum(ADNI_TEMP['Month'] == 6), ADNI_TEMP.shape
# Dementia
Final_Samples_Dementia = [[x, 1] for x in Final_Samples_Dementia]
Final_Samples_NonDementia = [[x, 0] for x in Final_Samples_NonDementia]

Final_Samples = Final_Samples_Dementia + Final_Samples_NonDementia
json.dump(Final_Samples, open('Final_Samples.json', 'w'))
# json.dump(Final_Samples, open('Final_Samples.json', 'w'))


Final_Samples_Dementia.__len__(), Final_Samples_NonDementia.__len__(), Final_Samples.__len__()

/tmp/ipykernel_10821/1350442401.py:2: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  ADNIMERGE = pd.read_csv('ADNIMERGE.csv')


   RID        PTID VISCODE  SITE COLPROT ORIGPROT  EXAMDATE DX_bl   AGE  \
0    2  011_S_0002      bl    11   ADNI1    ADNI1  08-09-05    CN  74.3   
1    3  011_S_0003      bl    11   ADNI1    ADNI1  12-09-05    AD  81.3   
2    3  011_S_0003     m06    11   ADNI1    ADNI1  13-03-06    AD  81.3   
3    3  011_S_0003     m12    11   ADNI1    ADNI1  12-09-06    AD  81.3   
4    3  011_S_0003     m24    11   ADNI1    ADNI1  12-09-07    AD  81.3   

  PTGENDER  ...  TAU_bl PTAU_bl   FDG_bl PIB_bl  AV45_bl  Years_bl  Month_bl  \
0     Male  ...     NaN     NaN  1.36665    NaN      NaN  0.000000   0.00000   
1     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.000000   0.00000   
2     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.498289   5.96721   
3     Male  ...   239.7   22.83  1.08355    NaN      NaN  0.999316  11.96720   
4     Male  ...   239.7   22.83  1.08355    NaN      NaN  1.998630  23.93440   

   Month     M update_stamp  
0      0   0.0      55:41.0  
1      0

(206, 1144, 1350)

In [25]:
import json
import pandas as pd
GWAS_ID = 'ieu-b-25'
usable_samples_ADNI = pd.read_csv(f'./PRSice_output/{GWAS_ID}/{GWAS_ID}.best', ' ')['IID'].to_numpy().tolist()
usable_samples_ADNI = {sample:idx for idx,sample in enumerate(usable_samples_ADNI)}
json.dump(usable_samples_ADNI, open('./usable_samples_ADNI.json', 'w'))

/home/naeem/anaconda3/envs/ad_venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3397: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [26]:
ALL_GWAS_IDS = [key for key in base_files]
print(ALL_GWAS_IDS)

['ukb-b-17006', 'ukb-b-19953', 'ukb-b-18275', 'ukb-b-8476', 'ieu-b-38', 'ieu-b-25', 'met-d-Total_C', 'ukb-b-6134', 'ukb-b-7663', 'ukb-a-257', 'ukb-b-17243', 'ukb-b-17627', 'ukb-b-770', 'ukb-b-14180', 'ukb-b-19732', 'ukb-d-20405_2', 'ieu-b-110', 'ukb-b-6358', 'ukb-b-15541', 'ukb-b-2209', 'ukb-b-4424', 'ukb-b-3957', 'ukb-d-20405_1', 'ieu-b-111', 'ukb-b-6324', 'ieu-b-39', 'ukb-b-20289', 'ieu-b-109', 'ukb-d-20405_0']


In [27]:
df

,GWAS_ID,Trait,Category
0,bbj-a-46,Pulse Pressure,Continuous
1,bbj-a-78,Smoking behaviors : Smoking initiation,Binary
2,ukb-b-13806,"Non-cancer illness code, self-reported: type 2...",Binary
3,ieu-a-1283,Alcohol consumption,Risk factor
4,ukb-d-20405_0,"Ever had known person concerned about, or reco...",Binary
5,ukb-d-20405_1,"Ever had known person concerned about, or reco...",Binary
6,ukb-d-20405_2,"Ever had known person concerned about, or reco...",Binary
7,met-d-Total_C,Total cholesterol,Continuous
8,ieu-b-109,HDL cholesterol,Continuous
9,ieu-b-110,LDL cholesterol,Continuous


In [28]:
df = pd.read_excel('GWAS_datasets_to_consider.xlsx', engine='openpyxl')[['GWAS_ID', 'Trait', 'Category']]
all_traits = []
for GWAS_ID in ALL_GWAS_IDS:
    trait = str(df[df['GWAS_ID'] == GWAS_ID]['Trait'].to_numpy()[0])
    print(GWAS_ID, ':', trait)
    all_traits += [trait]
print(all_traits)

ukb-b-17006 : Non-cancer illness code, self-reported: head injury
ukb-b-19953 : Body mass index (BMI)
ukb-b-18275 : Hearing difficulty/problems with background noise
ukb-b-8476 : Loneliness, isolation
ieu-b-38 : systolic blood pressure
ieu-b-25 : Cigarettes per Day
met-d-Total_C : Total cholesterol
ukb-b-6134 : Age completed full time education
ukb-b-7663 : Types of physical activity in last 4 weeks: Strenuous sports
ukb-a-257 : Hearing difficulty/problems: Yes
ukb-b-17243 : Non-cancer illness code, self-reported: anxiety/panic attacks
ukb-b-17627 : Non-oily fish intake
ukb-b-770 : Other meat intake
ukb-b-14180 : Mood swings
ukb-b-19732 : Non-cancer illness code, self-reported: hypothyroidism/myxoedema
ukb-d-20405_2 : Ever had known person concerned about, or recommend reduction of, alcohol consumption: Yes, during the last year
ieu-b-110 : LDL cholesterol
ukb-b-6358 : Non-cancer illness code, self-reported: stroke
ukb-b-15541 : Diagnoses - secondary ICD10: E66.9 Obesity, unspecified
u

In [29]:
# import torch
# torch.__version__

In [30]:
# # this is for demo purpose
# prsice_command = f'Rscript ./PRSice_linux/PRSice.R --dir ./PRSice_linux	\
# --prsice ./PRSice_linux/PRSice_linux     \
# --base ./PRSice_linux/simple_table_with_pval.txt     \
# --target ./PRSice_linux/ADNI_plink     \
# --thread 1     \
# --beta     \
# --binary-target F	\
# --out ./PRSice_output/PRSice	\
# --snp ID --chr CHROM --bp POS --A2 REF --A1 ALT --stat UKB-b-6134:ES --pvalue PVAL_generated_from_LP	\
# --extract ./PRSice_output/PRSice.valid'

In [31]:
#     # this is for demo purpose
#     import os
#     return_status = os.system(prsice_command) # generate the file named {GWAS_ID}.valid
#     print('1a:', return_status)